In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_sett(points, v):

    def calculate_distances(x, y):
        X, Y = np.meshgrid(x, y)
        distances = np.sqrt((X - X.T)**2 + (Y - Y.T)**2)
        return distances
    
    def calculate_diff(x):
        X, X_T = np.meshgrid(x, x)
        x_differences = X - X_T
        return x_differences

    x = points[:, 0]
    y = points[:, 1]
    
    dist = calculate_distances(x, y) + np.eye(v.shape[0])
    diff = calculate_diff(v) + np.eye(v.shape[0])
    result = np.abs(diff/dist) - np.eye(v.shape[0])
    
    return result
            
# Example usage:
x = np.array([1, 2, 3, 4, 5])
y = np.array([1, 2, 3, 4, 5])
volume = np.array([10, 20, 30, 40, 50])

rel_diff = get_sett(np.array([x, y]).T, volume)
print("Relative difference of sedimentation rates:")
print(rel_diff)    

Relative difference of sedimentation rates:
[[0.         7.07106781 7.07106781 7.07106781 7.07106781]
 [7.07106781 0.         7.07106781 7.07106781 7.07106781]
 [7.07106781 7.07106781 0.         7.07106781 7.07106781]
 [7.07106781 7.07106781 7.07106781 0.         7.07106781]
 [7.07106781 7.07106781 7.07106781 7.07106781 0.        ]]


In [3]:
list_df = pd.read_excel("uz_result_cube.XLSX", sheet_name=None)

In [4]:
list_df_new = {}
for key in list_df.keys():
    df_now = list_df[key].copy()
    df_now = df_now.query("X <= 25 & X >= 0")
    df_now = df_now.query("Y <= 25 & Y >= 0")
    df_now = df_now.query("Z == 0")
    df_now["group1"] = df_now["X"]//0.1
    df_now["group2"] = df_now["Y"]//0.1
    df_now["X"] = df_now["group1"]/10
    df_now["Y"] = df_now["group2"]/10
    df_now = df_now.groupby(by=["group1", "group2"]).mean()
    list_df_new[key] = df_now.loc[:, ["X", "Y", "Z", "u_z"]]

In [5]:
list_df_new["uk_all"].query("Z == 0")

X     Y    Z       u_z
group1 group2                           
0.0    0.0      0.0   0.0  0.0 -0.114764
       8.0      0.0   0.8  0.0 -0.120533
       17.0     0.0   1.7  0.0 -0.126194
       26.0     0.0   2.6  0.0 -0.131677
       35.0     0.0   3.5  0.0 -0.136899
...             ...   ...  ...       ...
249.0  214.0   24.9  21.4  0.0 -0.131933
       223.0   24.9  22.3  0.0 -0.126835
       232.0   24.9  23.2  0.0 -0.121468
       241.0   24.9  24.1  0.0 -0.115915
       249.0   24.9  24.9  0.0 -0.110248

[973 rows x 4 columns]

In [6]:
for key in list_df_new.keys():
    points = list_df_new[key].query("Z == 0").loc[:, ["X", "Y"]].to_numpy()
    u = list_df_new[key].query("Z == 0")["u_z"].to_numpy()

    #pd.DataFrame(get_sett(points, u)).isna().sum()
    sett = get_sett(points, u)
    print(key, round(np.mean(sett), 6), round(np.max(sett), 6))

uk_all 0.002262 0.009965
geo 0.001832 0.007847
uk 0.00225 0.009836
poly 0.002263 0.009842
